# Advanced pandas

## 12.1 Categorical Data

### Background and Motivation

In [ ]:
import numpy as np; import pandas as pd

In [ ]:
values = pd.Series(['apple','orange','apple','apple']*2)

In [ ]:
values

In [ ]:
pd.unique(values)

In [ ]:
pd.value_counts(values)

In [ ]:
values = pd.Series([0,1,0,0]*2)

In [ ]:
dim = pd.Series(['apple','orange'])

In [ ]:
values

In [ ]:
dim

In [ ]:
dim.take(values)

In [ ]:
pd.Series(['Volvo','BMW']).take([0,1,1,0]*2)

### Categorical Type in pandas

In [ ]:
fruits = ['apple','orange','apple','apple'] * 2

In [ ]:
N = len(fruits)

In [ ]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3,15,size=N),
                   'weight': np.random.uniform(0,4,size=N)
                  }, columns=['fruit','basket_id','count','weight'])

In [ ]:
df

In [ ]:
df['fruit']

In [ ]:
fruit_cat = df['fruit'].astype('category')

In [ ]:
fruit_cat

In [ ]:
c = fruit_cat.values

In [ ]:
c

In [ ]:
type(c)

In [ ]:
c.categories

In [ ]:
c.codes

In [ ]:
df['fruit'] = df['fruit'].astype('category')

In [ ]:
df.fruit

In [ ]:
my_c = pd.Categorical(['foo','bar','baz','foo','bar'])

In [ ]:
my_c

In [ ]:
categories = ['foo','bar','baz']

In [ ]:
codes = [0,1,2,0,0,1]

In [ ]:
my_cats2 = pd.Categorical.from_codes(codes, categories)

In [ ]:
my_cats2

In [ ]:
ordered_cats = pd.Categorical.from_codes(codes, categories, ordered=True)

In [ ]:
ordered_cats

In [ ]:
my_cats2.as_ordered()

### Computations With Categorical

In [ ]:
np.random.seed(12345)

In [ ]:
draws = np.random.randn(1000)

In [ ]:
draws[:5]

In [ ]:
bins = pd.qcut(draws, 4)

In [ ]:
bins

In [ ]:
bins = pd.qcut(draws, 4, labels=['Q1','Q2','Q3','Q4'])

In [ ]:
bins

In [ ]:
bins.codes[:10]

In [ ]:
bins.categories

In [ ]:
bins = pd.Series(bins, name='quartile')

In [ ]:
results = (pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index())

In [ ]:
results

In [ ]:
results['quartile']

In [ ]:
results['count']

#### Better performance with categoricals

In [ ]:
N = 10000000

In [ ]:
draws = pd.Series(np.random.randn(N))

In [ ]:
labels = pd.Series(['foo','bar','baz','qux']*(N//4))

In [ ]:
categories = labels.astype('category')

In [ ]:
labels.memory_usage()

In [ ]:
categories.memory_usage()

In [ ]:
%time _ = labels.astype('category')

## Categorical Methods

In [ ]:
s = pd.Series(['a','b','c','d']*2)

In [ ]:
cat_s = s.astype('category')

In [ ]:
cat_s

In [ ]:
cat_s.cat.codes

In [ ]:
cat_s.cat.categories

In [ ]:
cat_s2 = cat_s.cat.set_categories(['a','b','c','d','e'])

In [ ]:
cat_s2

In [ ]:
cat_s.value_counts()

In [ ]:
cat_s2.value_counts()

In [ ]:
cat_s.isin(['a','b'])

In [ ]:
cat_s3 = cat_s[cat_s.isin(['a','b'])]

In [ ]:
cat_s3

In [ ]:
cat_s3.cat.remove_unused_categories()

#### Table 12-1. Categorical methods for Series in pandas

![Table%2012-1.%20Categorical%20methods%20for%20Series%20in%20pandas](images/Table%2012-1.%20Categorical%20methods%20for%20Series%20in%20pandas.png)

### Creating dummy variables for modeling

In [ ]:
cat_s = pd.Series(['a','b','c','d']*2,dtype='category')

In [ ]:
cat_s

In [ ]:
pd.get_dummies(cat_s)

## 12.2 Advanced GroupBy Use

### Group Transforms and "unwrapped" GroupBys

In [ ]:
df = pd.DataFrame({'key':['a','b','c']*4, 'value':np.arange(12.)})

In [ ]:
df

In [ ]:
g = df.groupby('key').value

In [ ]:
g.mean()

In [ ]:
g.transform(lambda x: x.mean())

In [ ]:
g.transform('mean')

In [ ]:
g.transform(lambda x: x*2)

In [ ]:
g.transform(lambda x: x.rank(ascending=False))

In [ ]:
def normalise(x):
    return (x-x.mean())/x.std()

In [ ]:
g.transform(normalise)

In [ ]:
g.apply(normalise)

In [ ]:
normalised = (df['value']- g.transform('mean'))/g.transform('std')

In [ ]:
normalised

### Grouped Time Resampling

In [ ]:
N = 15

In [ ]:
times = pd.date_range('2018-02-25 00:00', freq='1min', periods=N)

In [ ]:
df = pd.DataFrame({'time':times, 'value':np.arange(N)})

In [ ]:
df

In [ ]:
df.set_index('time').resample('5min').count()

In [ ]:
df2 = pd.DataFrame({'time':times.repeat(3), 'key':np.tile(['a','b','c'], N), 'value':np.arange(N*3.)})

In [ ]:
df2[:7]

In [ ]:
time_key = pd.TimeGrouper('5min')

In [ ]:
resampled = (df2.set_index('time').groupby(['key', time_key]).sum())

In [ ]:
resampled

In [ ]:
resampled.reset_index()

## 12.3 Techiniques for Method Chaining

```python
# Usual - non-function way
df2 = df.copy()
df2.k=v

# Functional assign way
df2 = df.assign(k=v)
```

### The pipe Method

```python
# a sequence of function calls
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)

# rewrited using calls to pipe
result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe(h, arg4=v4))
```